# The DFT as a sampled DTFT
The discrete Fourier transform (DFT) can be considered as a sampled version of the discrete-time Fourier transform of one period of the signal.  Here we plot the DTFT, and superimpose a plot with $N$ samples of the DTFT.  If $N$ is too small, then it becomes clear that the samples cannot fully represent the DTFT, and hence information would be lost.

### Preamble
Start by importing the Python libraries that we will require

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

And define a function that will return true if running in a Jupyter Notebook

In [ ]:
def is_jupyter():
    """Return true if running in a Jupyter Notebook"""
    try:
        if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
            return True
        else:
            return False
    except: 
        return False

### User specified parameters
The following parameters can be specified.  

Parameter | Meaning
--------- | -------
<code>x_lims</code> |Set the range of the frequency axis in the plots.

In [ ]:
x_lims = [-1.5*np.pi, 2.9*np.pi]

### Function definitions
We start by defining the discrete-time Fourier transform for the sequence $$[-0.25,0,0.5,\underset{\uparrow}{1.5},0.5,0,-0.25]$$
Note that $\uparrow$ denotes the sample x(0).  We create a function that returns the frequency profile for a specified set of frequencies, <code>sample_points</code>.  In this example, the input has 7 points.

In [ ]:
def frequency_profile(sample_points):
    """
       Get y-axis data for sampling data x_vals.
    """
    X = 1.5 + np.cos(sample_points) - 0.5 * np.cos(3*sample_points)
    
    return X

We now define a function that will plot the discrete-time Fourier transform using a continuous line, and a set of <code>N_points</code> of the function that form the DFT values.

In [ ]:
def plot_figures(DTFT_freq, DTFT_amp, DFT_freq, DFT_amp):
    """
        Plot a figure showing a continuous DTFT, and samples of this
        function representing the DFT.  Also plot a second figure that
        displays the inverse DFT of the samples.  Arguments are:
            DTFT_freq: the frequencies corresponding to the DTFT plot samples
            DTFT_amp: the amplitude of the DTFT at the plot sampling points
            DFT_freq: the N_points frequencies of the DFT samples
            DFT_amp: the DFT amplitude at the N_points sampling points
    """

    # Extract the plot information that we need from the data
    N_points = len(DFT_freq)
    x_lims = [min(DTFT_freq),max(DTFT_freq)]
    
    # Create the plot figure
    plt.figure(figsize = (16, 8))
    # Update label font size
    plt.rcParams.update({'font.size': 16})
    
    # plot X in continuous frequency
    plt.plot(DTFT_freq, DTFT_amp)
    
    # Get current axes instance
    ax = plt.gca()
    
    # Get current ylim
    yll, yul = ax.get_ylim()
    
    # Plot sampling using stem 
    (markerLines, stemLines, baseLines) = plt.stem(DFT_freq, 
                                                   DFT_amp, 
                                                   markerfmt='ro',
                                                   use_line_collection = True,
                                                   bottom = yll)
    
    # Set stemlines and baselines color to red and linewidth = 1
    plt.setp(stemLines, color = 'red', linewidth=1)
    plt.setp(baseLines, color = 'red', linewidth=1)
    
    # Tidy up the plot to control axes sizes and labels
    plt.xlim(x_lims)
    plt.ylim([yll, yul])
    
    # Set y axis to invisible
    ax.get_yaxis().set_visible(False)
    plt.title('$X(\omega)$, N=%d'%N_points)
    plt.xlabel('Frequency (radians/sampling interval)')
    plt.xticks(np.linspace(-np.pi, 2*np.pi, 4), 
               ['$-\pi$', '0', '$\pi$', '$2\pi$'])
    
    # Save figure in python or ipython system
    if not is_jupyter(): plt.savefig('sampling_in_frequency_%d.pdf'%N_points)
    
    # Now use the inverse DFT transform to attempt to recover the data
    # from the samples of the DTFT

    # Calculate inverse DFT which should be real as the transform
    # is symmetric and real
    xn = np.real(np.fft.ifft(DFT_amp))

    # Create the second plot figure
    plt.figure(figsize = (16, 8))
    # Update label font size
    plt.rcParams.update({'font.size': 16})
    plot_data = np.tile(xn,3)
    plot_time = np.arange(-len(xn),2*len(xn))
    # Plot sampling using stem 
    (markerLines, stemLines, baseLines) = plt.stem(plot_time, 
                                                   plot_data, 
                                                   markerfmt='ro',
                                                   use_line_collection = True,
                                                   bottom = 0)

    # Set stemlines and baselines color and linewidth
    plt.setp(stemLines, color = 'blue', linewidth=1)
    plt.setp(baseLines, color = 'black', linewidth=1)

    plt.xlim([-len(xn),2*len(xn)-1])
    
    plt.title('Inverse DFT, $x(n)$, N=%d'%N_points)
    plt.xlabel('Time (samples)')
    plt.ylabel('Amplitude')

    # Save figure in python or ipython system
    if not is_jupyter(): plt.savefig('sampling_in_frequency_inverse_transform_%d.pdf'%N_points)


Here we generate the continuous and discrete data that is to be plotted.  The function computes the sample spacing (which is very closely spaced for the DTFT plot), and calls the function to determine the value at each of these frequency samples.  These are passed to <code>plot_figure()</code> to display the result.

In [ ]:
def plot_sampled_DTFT(x_lims, N_points):
    """
       Plot figure with different N points.
    """
    DTFT_freq = np.linspace(x_lims[0], x_lims[1],
                     int((x_lims[1]-x_lims[0])/0.01*np.pi))
    DTFT_amp = frequency_profile(DTFT_freq)
    
    # Define x-axis and y-axis for sampling
    DFT_freq = np.arange(0,N_points) * 2 * np.pi / N_points
    DFT_amp = frequency_profile(DFT_freq)

    plot_figures(DTFT_freq, DTFT_amp, DFT_freq, DFT_amp)

### Plot the figure for different numbers of samples in the frequency domain

In [ ]:
# For the first plot, use 8 points
plot_sampled_DTFT(x_lims, 8)

Now let's plot the graphs for different values of $N$.  When $N$ is sufficiently large, then the original data sequence can be recovered.  However, if $N$ is too small, then it is not possible to use the inverse transform to recover this data from the frequency domain samples.

In [ ]:
# For the remaining plots, repeat with different N 
plot_sampled_DTFT(x_lims, 11)
plot_sampled_DTFT(x_lims, 7)
plot_sampled_DTFT(x_lims, 4)

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.